In [1]:
import numpy as np
import timeit
import pandas as pd


## Definições

$$
\lim_{x1 \to 0} f(x1) = \infty \text{  e  } \lim_{x2 \to 0} f(x2) = \infty
$$

$$
f(x) = 3^{\frac{1}{x_1+x_2}} + x_1^2 + x_2^2 \\
\nabla f(x) = [2 \dot x_1 -\frac{ e^\frac{ln(3)}{x_1+x_2}}{(x_1+x_2)**2}, 2 \dot x_2 -\frac{ e^\frac{ln(3)}{x_1+x_2}}{(x_1+x_2)**2}] \\

$$

In [2]:
def f(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return 3**(1/(x1+x2))+x1**2+x2**2

In [3]:
def Gradiente(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2*x1-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2],
            [2*x2-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2]])

In [4]:
def Heissiana(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))))/(x1+x2)**4, -np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4],

            [-np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4 ,2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))*(x1+x2)))/(x1+x2)**4]])

In [5]:
def definida_positiva_2x2(matrix):    
    if matrix[0][0] > 0 and np.linalg.det(matrix)>0:
        return True
    else: 
        return False

In [6]:
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

In [7]:
def test_combinations(search, method, maxiter, tol):
    result = {
        "start": [],
        "gamma": [],
        "eta": [],
        "Armijo Iterations": [],
        "Method Iterations": [],
        "Optical Point Found": [],
        "Optical Value Found": [],
        "Time": []
    }
    for point in search['start']:
        for gamma in search['gamma']:
            for eta in search['eta']:
                start = timeit.default_timer()
                met =  method(point, tol, maxiter, gamma, eta)
                stop = timeit.default_timer()
                result["start"].append(point)
                result["gamma"].append(gamma)
                result["eta"].append(eta)
                result["Armijo Iterations"].append(met[2])
                result["Method Iterations"].append(met[1])
                result["Optical Point Found"].append(met[0])
                result["Optical Value Found"].append(f(met[0]))
                result["Time"].append(stop - start)

    return result

In [8]:
search = {
    "start": [
        [[0.5], [0.5]], 
        [[-1], [-1]], 
        [[-6000], [-5000]],
        [[6000],[5000]],
        [[5], [-0.5]],
        [[-5], [0.5]] 
    ],
    "gamma": [
        0.8,
        0.65,
        0.5
    ],
    "eta": [
        0.25
    ],
    "crit parada": [
        "gradiente",
        "x",
        "f"
    ]
}

## Implementação dos métodos

### Busca de Armijo

In [9]:
def Armijo(x,gama,d,n):
    t = 1
    k = 0
    d = np.array(d)
    x = np.array(x)
    while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):
        t=gama*t
        k = k+1
    return [t,k]

### Método do Gradiente

In [169]:
def met_gradiente(x,tol,maxiter,gama,n, debug):
    x = np.array(x)
    m = 0
    k = 0
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
    return [x,m,k] 

In [170]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_gradiente, maxiter, tol)       
grad_res = pd.DataFrame.from_records(result)    
grad_res

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_9684\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_9684\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,73,9,"[[0.6763370143192518], [0.6763370143192518]]",3.167673e+00,0.004754,0.25,0.80,"[[0.5], [0.5]]"
1,30,7,"[[0.6763255367588685], [0.6763255367588685]]",3.167673e+00,0.001662,0.25,0.65,"[[0.5], [0.5]]"
2,13,4,"[[0.6763349644096142], [0.6763349644096142]]",3.167673e+00,0.000679,0.25,0.50,"[[0.5], [0.5]]"
3,26,6,"[[-8.873752021730419e-06], [-8.873752021730419...",1.574869e-10,0.001518,0.25,0.80,"[[-1], [-1]]"
4,13,6,"[[-1.1955435295191822e-05], [-1.19554352951918...",2.858649e-10,0.000851,0.25,0.65,"[[-1], [-1]]"
5,22,11,"[[-1.8035048834030365e-05], [-1.80350488340303...",6.505260e-10,0.001595,0.25,0.50,"[[-1], [-1]]"
6,34,13,"[[-3.446487932660803e-05], [1.7669996772328288...",1.500057e-09,0.002079,0.25,0.80,"[[-7000], [-5000]]"
7,20,13,"[[-3.3722409913135255e-05], [1.497763437957704...",1.361530e-09,0.001582,0.25,0.65,"[[-7000], [-5000]]"
8,52,26,"[[-2.5718087667200485e-05], [4.084234720494812...",6.781010e-10,0.005722,0.25,0.50,"[[-7000], [-5000]]"
9,38,13,"[[-1.3598002875922862e-05], [-4.17201727723524...",2.023114e-10,0.002195,0.25,0.80,"[[7000], [5000]]"


#### Conclusões
1. O ponto (0.5, 0.5) é próximo do mínimo local e por consequência converge para o mínimo local
2. O ponto (-1, -1) é próximo de (0, 0) e por consequência converge para valores próximos de (0, 0)
3. O ponto (5, -0.5), que pertencem a segunda parte da função, para passos grandes pula e converge pra próximo de (0, 0) e para passos pequenos converge para o mínimo local
4. O ponto (-5, 0.5), que pertencem a primeira parte da função, para passos grandes pula e converge para o mínimo local e para passos pequenos converge para próximo de (0, 0)

### Método de Newton

In [54]:
def met_newton(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k = 0
    df = 1
    inv = 1
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - (np.linalg.inv(Heissiana(x)))@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
        k = k+1
        if debug:
            df *= definida_positiva_2x2(Heissiana(x))
            inv *=  is_invertible(Heissiana(x))
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", Heissiana(x))
            print("Definida Positiva:\n", definida_positiva_2x2(Heissiana(x)))
            print("Singular:\n", not is_invertible(Heissiana(x)))
    if debug:
        return [x,m,k, df, inv] 
    else: 
        return [x,m,k] 

In [50]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_newton, maxiter, tol)       
newton_res = pd.DataFrame.from_records(result)   
newton_res 

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.005126,0.25,0.80,"[[0.5], [0.5]]"
1,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.004125,0.25,0.65,"[[0.5], [0.5]]"
2,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.001849,0.25,0.50,"[[0.5], [0.5]]"
3,154962,1000,"[[-0.06787345025363754], [-0.14197059824577488]]",3.008728e-02,4.243054,0.25,0.80,"[[-1], [-1]]"
4,90775,1000,"[[-0.3797911180521609], [-0.2960744738152126]]",4.287167e-01,3.234556,0.25,0.65,"[[-1], [-1]]"
5,26,14,"[[-2.5530470568192873e-05], [-2.45032780931528...",1.252216e-09,0.003868,0.25,0.50,"[[-1], [-1]]"
6,162137,1000,"[[-0.3594177907609244], [-0.309642320351908]]",4.186475e-01,5.897865,0.25,0.80,"[[-6000], [-5000]]"
7,90108,1000,"[[-0.4237923470116344], [0.26260390832536934]]",2.496572e-01,2.585883,0.25,0.65,"[[-6000], [-5000]]"
8,29,16,"[[-1.5747819714917455e-14], [-8.68402572073989...",3.234061e-28,0.003261,0.25,0.50,"[[-6000], [-5000]]"
9,18,12,"[[0.6763342609151929], [0.6763225743104986]]",3.167673e+00,0.002439,0.25,0.80,"[[6000], [5000]]"


#### Análise dos resultados
 Newton não parece se comportar tão bem como o grandiente quando se trata da descontinuidade. Hipotese: a Hessiana nem sempre fica definida positiva próximo das descontinuidades, ou seja, as direções escolhidas não são de descida.

##### 1. Ponto (-5, -0.5) com gamma 0.8

In [55]:
newt_1 = met_newton([[-5], [-0.5]], 0.0001, 1000, 0.8, 0.25, True)
newt_1[3], newt_1[4]


Iteração:  1
--------------------
Ponto:
 [[-0.99482541]
 [-0.09221935]]
Hessiana:
 [[2.88737271 0.88737271]
 [0.88737271 1.69200757]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[-0.14682474]
 [-0.22794649]]
Hessiana:
 [[11.20124722  9.20124722]
 [ 9.20124722  3.03655357]]
Definida Positiva:
 False
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.

(0, 1)

- Como a Hessiana não é definida positiva nas iterações finais de (-5, -0.5) a direção  escolhida nas últimas iterações não é descida e não há convergência

##### 2. Ponto (-1, -1) com gamma 0.5

In [30]:
newt_2 = met_newton([[-1], [-1]], 0.0001, 1000, 0.5, 0.25, True)
newt_2[3], newt_2[4]


Iteração:  1
--------------------
Ponto:
 [[-0.52291624]
 [-0.45851883]]
Hessiana:
 [[3.19789532 1.19789532]
 [1.19789532 1.66588578]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[-0.31276487]
 [ 0.16437405]]
Hessiana:
 [[6.2764348  4.2764348 ]
 [4.2764348  3.10661683]]
Definida Positiva:
 True
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[-0.1416903 ]
 [-0.07712231]]
Hessiana:
 [[11.79987762  9.79987762]
 [ 9.79987762  4.09048178]]
Definida Positiva:
 False
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[-0.11621509]
 [-0.06342811]]
Hessiana:
 [[9.21822256 7.21822256]
 [7.21822256 3.7222609 ]]
Definida Positiva:
 False
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[-0.09860269]
 [-0.04330514]]
Hessiana:
 [[5.64581867 3.64581867]
 [3.64581867 2.95868747]]
Definida Positiva:
 True
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[-0.06176116]
 [-0.07006205]]
Hessiana:
 [[4.70795647 2.7079

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


(0, 1)

- A Hessiana não é definida positiva em todas as iterações, mas Newton converge

##### 3. Ponto (6000, 5000) com gamma 0.8

In [31]:
newt_3 = met_newton([[6000], [5000]], 0.0001, 1000, 0.8, 0.35, True)
newt_3[3], newt_3[4]


Iteração:  1
--------------------
Ponto:
 [[1200.00000769]
 [1000.02481245]]
Hessiana:
 [[2.00000000e+00 1.45384545e-13]
 [1.45384545e-13 2.00000000e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[240.00000163]
 [200.00496266]]
Hessiana:
 [[2.00000000e+00 9.10470209e-11]
 [9.10470209e-11 2.00000003e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[48.00000262]
 [40.00099689]]
Hessiana:
 [[2.00000006e+00 5.74755377e-08]
 [5.74755377e-08 2.00000329e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[9.60006  ]
 [8.0003105]]
Hessiana:
 [[2.00003776e+00 3.77600919e-05]
 [3.77600919e-05 2.00044235e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[1.92178772]
 [1.60313193]]
Hessiana:
 [[2.03011421 0.03011421]
 [0.03011421 2.07919163]]
Definida Positiva:
 True
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[0

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


(1, 1)

- Newton converge e a Hessiana é definida positiva em todas as iterações, ou seja, todas as iterações são de descida

## Método Quase Newton


#### DFP

In [32]:
def DFP(x_atual,x_antigo,H):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)
    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if np.linalg.norm(p) != 0 and np.linalg.norm(q) != 0:
        return H + (p@np.transpose(p))/(np.transpose(p)@q) - (H@q@np.transpose(q)@H)/(np.transpose(q)@H@q)
    else: 
        print('pq <= 0')
        return H

In [33]:
def met_quase_newton_DFP(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k =0
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        H = DFP(x,x_antigo,H)
        m = m + 1
        k = k + ar[1]
        
    return [x,m,k] 

In [43]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_quase_newton_DFP, maxiter, tol)       
dfp_res = pd.DataFrame.from_records(result)    
dfp_res

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,10,5,"[[0.6763353628281227], [0.6763353628281227]]",3.167673e+00,0.004534,0.25,0.80,"[[0.5], [0.5]]"
1,6,4,"[[0.676332691443663], [0.676332691443663]]",3.167673e+00,0.001129,0.25,0.65,"[[0.5], [0.5]]"
2,4,5,"[[0.6763326473319318], [0.6763326473319318]]",3.167673e+00,0.001082,0.25,0.50,"[[0.5], [0.5]]"
3,10,6,"[[-3.516154439006929e-05], [-3.516154439009241...",2.472668e-09,0.002788,0.25,0.80,"[[-1], [-1]]"
4,3,4,"[[-6.071532165918825e-18], [-1.040834085586084...",1.451971e-34,0.001423,0.25,0.65,"[[-1], [-1]]"
5,3,5,"[[5.2888737226558513e-17], [-5.532141585107286...",5.857678e-33,0.001617,0.25,0.50,"[[-1], [-1]]"
6,11,13,"[[0.676335010060589], [0.6763313704115276]]",3.167673e+00,0.004120,0.25,0.80,"[[-6000], [-5000]]"
7,10,13,"[[0.6763277833164886], [0.6763333982829317]]",3.167673e+00,0.010248,0.25,0.65,"[[-6000], [-5000]]"
8,18,20,"[[-3.7239897350224685e-05], [1.786778325142697...",1.706068e-09,0.007404,0.25,0.50,"[[-6000], [-5000]]"
9,11,11,"[[-1.2772953068963789e-05], [8.839925742881646...",2.412926e-10,0.007605,0.25,0.80,"[[6000], [5000]]"


- Os resultads deste método ficaram semelhantes ao método de newton

#### BFGS

In [10]:
def BFGS(x_atual,x_antigo,H, debug):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)

    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if debug:
        print("Vetor p: ",p)
        print("Vetor q: ",p)
    return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)

In [88]:
def met_quase_newton_BFGS(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k =0
    df = 1
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        if debug:
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", H)
            print("Definida Positiva:\n", definida_positiva_2x2(H))
        H = BFGS(x,x_antigo,H, debug)
        m = m + 1
        k = k + ar[1]
    if debug:
        return [x,m,k,df] 
    else:
        return [x,m,k]  


In [146]:
x = 0.4
x_antigo = x
x += 5
x, x_antigo

(5.4, 0.4)

In [149]:
def met_quase_newton_BFGS_mod(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k =0
    df = 1
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        if debug:
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", H)
            print("Definida Positiva:\n", definida_positiva_2x2(H))
        if ((x - x_antigo) < tol).prod():
            x = x + tol
            print("Novo ponto:\n" ,x)
            
        H = BFGS(x,x_antigo,H, debug)
        m = m + 1
        k = k + ar[1]
    if debug:
        return [x,m,k,df] 
    else:
        return [x,m,k] 

##### Análise do Erro do método
No primeiro teste que fizemos a implementação dá erro, pois p ou q são iguais a zero.

Teorema: Quando a hessiana é definida positiva e o t é calculado por busca exata, p*q>0 e a próxima hessiana cálculada é definida positiva.

O problema ta na busca armijo ou na hessiana?

In [141]:
met_quase_newton_BFGS([[0.5], [0.5]], 0.0001, 1000, 0.8, 0.25)

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: invalid value encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)


[array([[nan],
        [nan]]),
 5,
 193]

In [148]:
met_quase_newton_BFGS_mod([[0.5], [0.5]], 0.0001, 1000, 0.80, 0.25, True)


Iteração:  0
--------------------
Ponto:
 [[0.74651361]
 [0.74651361]]
Hessiana:
 [[1. 0.]
 [0. 1.]]
Definida Positiva:
 True
Vetor p:  [[0.24651361]
 [0.24651361]]
Vetor q:  [[0.24651361]
 [0.24651361]]

Iteração:  1
--------------------
Ponto:
 [[-0.05667853]
 [-0.05667853]]
Hessiana:
 [[1.36483979 0.36483979]
 [0.36483979 1.36483979]]
Definida Positiva:
 True
Novo ponto:
 [[-0.05657853]
 [-0.05657853]]
Novo ponto:
 [[-0.05557853]
 [-0.05557853]]
Novo ponto:
 [[-0.04557853]
 [-0.04557853]]
Novo ponto:
 [[0.05442147]
 [0.05442147]]
Novo ponto:
 [[1.05442147]
 [1.05442147]]
Vetor p:  [[0.30790786]
 [0.30790786]]
Vetor q:  [[0.30790786]
 [0.30790786]]

Iteração:  2
--------------------
Ponto:
 [[-0.55451836]
 [-0.55451836]]
Hessiana:
 [[ 0.97519423 -0.02480577]
 [-0.02480577  0.97519423]]
Definida Positiva:
 True
Novo ponto:
 [[-0.55441836]
 [-0.55441836]]
Novo ponto:
 [[-0.55341836]
 [-0.55341836]]
Novo ponto:
 [[-0.54341836]
 [-0.54341836]]
Novo ponto:
 [[-0.44341836]
 [-0.44341836]]

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2



Iteração:  15
--------------------
Ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Hessiana:
 [[9.62376198e+192 9.62376198e+192]
 [9.62376198e+192 9.62376198e+192]]
Definida Positiva:
 False
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075512e+12]
 [-4.55075512e+12]]
Novo ponto:
 [[-4.55075511e+12]
 [-4.55075511e+12]]
Novo ponto:
 [[-4.55075501e+12]
 [-4.55075501e+12]]
Novo ponto:
 [[-4.55075401e+12]
 [-4.55075401e+12]]
Novo ponto:
 [[-4.55074401e+12]
 [-4.55074401e+12]]
Novo ponto:
 [[-4.55064401e+12]
 [-4.55064401e+12]]
Novo ponto:
 [[-4.54964401e+12]
 [-4.54964401e+12]]
Novo ponto:
 [[-4.53964401e+12]
 [-4.53964401e+12]]
Novo ponto:
 [[-4.43964401e+1

c:\Users\mwuillau\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\linalg\linalg.py:2146: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3290355684.py:6: RuntimeWarning: overflow encountered in matmul
  while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):



Iteração:  17
--------------------
Ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Hessiana:
 [[5.25345821e+252 5.25345821e+252]
 [5.25345821e+252 5.25345821e+252]]
Definida Positiva:
 False
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855815e+14]
 [-2.66855815e+14]]
Novo ponto:
 [[-2.66855814e+14]
 [-2.66855814e+14]]
Novo ponto:
 [[-2.66855804e+14]
 [-2.66855804e+14]]
Novo ponto:
 [[-2.66855704e+14]
 [-2.66855704e+14]]
Novo ponto:
 [[-2.66854704e+14]
 [-2.66854704e+14]]
Novo ponto:
 [[-2.66844704e+14]
 [-2.66844704e+14]]
Novo ponto:
 [[-2.66744704e+1

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3290355684.py:6: RuntimeWarning: invalid value encountered in multiply
  while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):



Iteração:  19
--------------------
Ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Hessiana:
 [[3.2076287e+288 3.2076287e+288]
 [3.2076287e+288 3.2076287e+288]]
Definida Positiva:
 False
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Vetor p:  [[2.]
 [2.]]
Vetor q:  [[2.]
 [2.]]

Iteração:  20
--------------------
Ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Hessiana:
 [[9.94364897e+289 9.94364897e+289]
 [9.94364897e+289 9.94364897e+289]]
Definida Positiva:
 False
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Novo ponto:
 [[1.08442553e+16]
 [1.08442553e+16]]
Vetor p:  [[10.]
 [10.]]
Vetor

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\1576954661.py:9: RuntimeWarning: overflow encountered in matmul
  d = -1*H@Gradiente(x)



Iteração:  21
--------------------
Ponto:
 [[-inf]
 [-inf]]
Hessiana:
 [[7.94497553e+292 7.94497553e+292]
 [7.94497553e+292 7.94497553e+292]]
Definida Positiva:
 False
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]
 [-inf]]
Novo ponto:
 [[-inf]


OverflowError: int too large to convert to float

In [130]:
met_quase_newton_BFGS_mod([[6000], [5000]], 0.0001, 1000, 0.5, 0.25, True)


Iteração:  0
--------------------
Ponto:
 [[-9.60393209e-06]
 [-3.10157395e-02]]
Hessiana:
 [[2.00000000e+00 1.74699505e-09]
 [1.74699505e-09 2.00001240e+00]]
Definida Positiva:
 True
Vetor p:  [[-6000.0000096 ]
 [-5000.03101574]]
Vetor q:  [[-6000.0000096 ]
 [-5000.03101574]]

Iteração:  1
--------------------
Ponto:
 [[ 0.01386079]
 [-0.01945701]]
Hessiana:
 [[2.88000733e+08 2.40002098e+08]
 [2.40002098e+08 2.00002990e+08]]
Definida Positiva:
 True
Vetor p:  [[0.01387039]
 [0.01155873]]
Vetor q:  [[0.01387039]
 [0.01155873]]

Iteração:  2
--------------------
Ponto:
 [[ 0.01386079]
 [-0.01945701]]
Hessiana:
 [[-2.87625185e+08 -2.39689145e+08]
 [-2.39689145e+08 -1.99742193e+08]]
Definida Positiva:
 False
Novo ponto:
 [[ 0.01396079]
 [-0.01935701]]
Vetor p:  [[1.e-04]
 [1.e-04]]
Vetor q:  [[1.e-04]
 [1.e-04]]

Iteração:  3
--------------------
Ponto:
 [[ 0.01651399]
 [-0.01705171]]
Hessiana:
 [[2.39689126e+08 2.43683670e+08]
 [2.43683670e+08 2.39689126e+08]]
Definida Positiva:
 False


C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\1474122156.py:5: RuntimeWarning: overflow encountered in exp
  return np.array([[2*x1-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2],
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\1474122156.py:6: RuntimeWarning: overflow encountered in exp
  [2*x2-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2]])
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: invalid value encountered in multiply
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: invalid value encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q

[array([[nan],
        [nan]]),
 8,
 369,
 1]

- Conclusão as Hessianas são definidas positivas o problema tá no uso da busca de armijo

In [126]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_quase_newton_BFGS, maxiter, tol)     
bfgs_res = pd.DataFrame.from_records(result)      
bfgs_res

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: invalid value encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: divide by zero encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_19132\551132956.py:10: RuntimeWarning: invalid value encountered in matmul
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@

,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,193,5,"[[nan], [nan]]",NaN,0.009242,0.25,0.80,"[[0.5], [0.5]]"
1,175,8,"[[0.6763260400321802], [0.6763260400321802]]",3.167673,0.010931,0.25,0.65,"[[0.5], [0.5]]"
2,169,12,"[[0.676330667239791], [0.676330667239791]]",3.167673,0.010872,0.25,0.50,"[[0.5], [0.5]]"
3,194,4,"[[nan], [nan]]",NaN,0.006803,0.25,0.80,"[[-1], [-1]]"
4,102,4,"[[nan], [nan]]",NaN,0.004015,0.25,0.65,"[[-1], [-1]]"
5,61,5,"[[nan], [nan]]",NaN,0.004463,0.25,0.50,"[[-1], [-1]]"
6,3347,12,"[[nan], [nan]]",NaN,0.128979,0.25,0.80,"[[-6000], [-5000]]"
7,1016,9,"[[nan], [nan]]",NaN,0.026819,0.25,0.65,"[[-6000], [-5000]]"
8,603,9,"[[nan], [nan]]",NaN,0.025238,0.25,0.50,"[[-6000], [-5000]]"
9,2907,11,"[[nan], [nan]]",NaN,0.100169,0.25,0.80,"[[6000], [5000]]"


##### Análise dos resultados
- Somente para o ponto próximo do mínimo local com passo pequeno o método converge

# Comparação dos Métodos